In [2]:
import requests
from bs4 import BeautifulSoup

In [4]:
# Getting a function with handling errors
def scrape_web(web_url):    
    try:
        data = {}
        # Getting Sections Data
        main = requests.get(URL)
        if main.status_code != 200:
            print(f'Error: {main.status_code}')

        main_soup = BeautifulSoup(main.text,'lxml')
        sections = main_soup.find('ul', attrs = {'class': 'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
        sections_links = [section.find('a').get('href') for section in sections]
        sections_names = [section.find('a').get_text() for section in sections]
        
        for section in sections:
            section_name = section.find('a').get_text()
            section_link = section.find('a').get('href')
            data[section_name] = {'link': section_link}
            section_articles = data[section_name]
            try:
                # Getting Articles Data
                section_req = requests.get(section_link)
                section_soup = BeautifulSoup(section_req.text,'lxml')
                articles = section_soup.find_all('article', attrs = {'class': 'article-item'})
                section_articles['articles'] = {}
                
                for i, article in enumerate(articles):
                    article_link = articles[i].find(attrs={'class': 'title-list'}).find('a').get('href')
                    article_link = 'https://www.pagina12.com.ar' + article_link
                    article_title = articles[i].find(attrs={'class': 'title-list'}).find('a').get_text()
                    
                    try:
                        # Getting Article Data
                        article_req = requests.get(article_link)
                        article_soup = BeautifulSoup(article_req.text,'lxml')
                        #print(article_soup)
                        # Extract Title
                        title = article_soup.find(attrs={'class':'article-header'}).find('h1').get_text()
                        # Extract Subtitle
                        subtitle = article_soup.find(attrs={'class':'article-header'}).find('h4').get_text()
                        # Extract Date
                        date = article_soup.find(attrs={'class':'article-info'}).find('time').get_text()
                        # Extract Image
                        image = article_soup.find(attrs={'class':'article-main-media-image__container'}).find('img').get('src')

                        # Putting all article data together
                        article_head = {article_title: {
                                                        'link': article_link,
                                                        'title': title,
                                                        'subtitle': subtitle,
                                                        'date': date}}
                        section_articles['articles'].update(article_head)
                    except Exception as e:
                        print(f'Could not open following article: {article_link}')
                        print(e)
                    
            except Exception as e:
                print(f'Could not open following section: {section_link}')
                print(e)
        
        return data
    
    except Exception as e:
        print(f'Could not open following web: {url}')
        print(e)

URL = 'https://www.pagina12.com.ar/'
scrape_web(URL)

{'El país': {'link': 'https://www.pagina12.com.ar/secciones/el-pais',
  'articles': {'Cristina Kirchner y la tapa de la revista Noticias': {'link': 'https://www.pagina12.com.ar/394133-cristina-kirchner-y-la-tapa-de-la-revista-noticias',
    'title': 'Cristina Kirchner y la tapa de la revista Noticias',
    'subtitle': 'Un tuit irónico con desmentida incluida',
    'date': '8 de enero de 2022 - 04:50'},
   'Los desafíos de conducir Celac': {'link': 'https://www.pagina12.com.ar/394098-los-desafios-de-conducir-celac',
    'title': 'Los desafíos de conducir Celac',
    'subtitle': 'Los expertos y el rol de la Argentina en la región',
    'date': '8 de enero de 2022 - 00:22'},
   'Alberto Fernández: "Vamos a promover el consenso y la pluralidad, sin exclusiones"': {'link': 'https://www.pagina12.com.ar/394097-alberto-fernandez-vamos-a-promover-el-consenso-y-la-pluralid',
    'title': 'Alberto Fernández: "Vamos a promover el consenso y la pluralidad, sin exclusiones"',
    'subtitle': 'La Arg